In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

> ### By Jiahui Tang, Tiffany Yang, Xin Zeng

### Preprocessing
FillNA and descriptively look at quasi columns

In [2]:
data = pd.read_csv('APCOMP221Pset1.csv', low_memory=False)

In [3]:
data.head()

,course_id,user_id,username,registered,viewed,explored,certified,completed,ip,cc_by_ip,...,roles_isInstructor,roles_isStaff,roles_isCCX,roles_isFinance,roles_isLibrary,roles_isSales,forumRoles_isAdmin,forumRoles_isCommunityTA,forumRoles_isModerator,forumRoles_isStudent
0,HarvardX/PH525.1x/1T2018,1488411,KIRSTEN SUAREZ,True,False,NaN,False,False,81.108.107.58,GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,HarvardX/PH525.1x/1T2018,7013084,CAREY FOSTER,True,True,True,False,False,205.175.107.76,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,HarvardX/PH525.1x/1T2018,4083257,CLAUDINE FARMER,True,True,False,False,False,103.212.146.137,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,HarvardX/PH525.1x/1T2018,4605571,SHEREE BONNER,True,True,False,False,False,172.221.204.94,US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,HarvardX/PH525.1x/1T2018,1499820,MITCHELL VALDEZ,True,True,True,False,False,193.225.200.92,HU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [4]:
quasi_lists = ['course_id', 'cc_by_ip', 'countryLabel', 'continent', 'city', 'region',
              'subdivision', 'postalCode', 'LoE', 'YoB', 'gender', 'nforum_posts', 'nforum_votes', 
              'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', 'nforum_events', 
              'mode', 'email_domain']

In [5]:
data[quasi_lists].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1141735 entries, 0 to 1141734
Data columns (total 20 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   course_id        1141735 non-null  object 
 1   cc_by_ip         957309 non-null   object 
 2   countryLabel     957540 non-null   object 
 3   continent        959097 non-null   object 
 4   city             827422 non-null   object 
 5   region           789588 non-null   object 
 6   subdivision      807161 non-null   object 
 7   postalCode       437268 non-null   object 
 8   LoE              975782 non-null   object 
 9   YoB              967016 non-null   float64
 10  gender           989616 non-null   object 
 11  nforum_posts     39360 non-null    float64
 12  nforum_votes     39360 non-null    float64
 13  nforum_endorsed  39360 non-null    float64
 14  nforum_threads   39360 non-null    float64
 15  nforum_comments  39360 non-null    float64
 16  nforum_pinned    3

> To identify which columns we need to remove, we plan to use the groupby function in Python. However, groupby function would ignore NaN values when using variables as groups. To ensure we get correct result, we first decide to replace NaN values as -999, which is a numerical value that has not been seen in our dataframe.

In [6]:
data_fillna = data.fillna(-999)

### Task 1 Suppression K-Anonymity

In [7]:
# supression
def supression(data, quasi_set, k):
    len_remove = 0
    lst_remove = []
    grouped = data.groupby(by=quasi_set)
    for name, group in grouped:
        if group.shape[0] < k:
            len_remove += group.shape[0]
            lst_remove.extend(list(group.index))
    return len_remove, lst_remove

In [8]:
# get completion rate for each course
def complete_rate(data):
    grouped = data.groupby(by=["course_id"])
    rate = grouped[["completed"]].sum() / grouped[["completed"]].count()
    return rate

In [9]:
len_remove, lst_remove = supression(data_fillna, quasi_lists, 3)
len_remain = data_fillna.shape[0] - len_remove
print(str(len_remain) + " records of the original edX dataset remain if we use suppression to make it 3-anonymous")
data_drop_3 = data_fillna.drop(lst_remove, axis=0)
rate_3 = complete_rate(data_drop_3)
print(rate_3)

141749 records of the original edX dataset remain if we use suppression to make it 3-anonymous
                            completed
course_id                            
HarvardX/1368.1x/2T2016      0.000000
HarvardX/1368.2x/2T2016      0.000000
HarvardX/1368.4x/2T2016      0.000000
HarvardX/1368x/2T2017        0.000000
HarvardX/AI12.1x/2013_SOND   0.000000
...                               ...
Harvardx/HLS2X/4T2017        0.008065
Harvardx/HLS2X/T12016        0.004024
VJx/VJx/3T2014               0.000000
VJx/VJx_2/3T2016             0.000000
VJx/VJx_S/3T2015             0.000000

[249 rows x 1 columns]


In [10]:
len_remove, lst_remove = supression(data_fillna, quasi_lists, 4)
len_remain = data_fillna.shape[0] - len_remove
print(str(len_remain) + " records of the original edX dataset remain if we use suppression to make it 4-anonymous")
data_drop_4 = data_fillna.drop(lst_remove, axis=0)
rate_4 = complete_rate(data_drop_4)
print(rate_4)

120995 records of the original edX dataset remain if we use suppression to make it 4-anonymous
                            completed
course_id                            
HarvardX/1368.1x/2T2016      0.000000
HarvardX/1368.4x/2T2016      0.000000
HarvardX/1368x/2T2017        0.000000
HarvardX/AI12.1x/2013_SOND   0.000000
HarvardX/AI12.2x/2013_SOND   0.000000
...                               ...
Harvardx/HLS2X/4T2017        0.010309
Harvardx/HLS2X/T12016        0.002370
VJx/VJx/3T2014               0.000000
VJx/VJx_2/3T2016             0.000000
VJx/VJx_S/3T2015             0.000000

[227 rows x 1 columns]


In [11]:
len_remove, lst_remove = supression(data_fillna, quasi_lists, 5)
len_remain = data_fillna.shape[0] - len_remove
print(str(len_remain) + " records of the original edX dataset remain if we use suppression to make it 5-anonymous")
data_drop_5 = data_fillna.drop(lst_remove, axis=0)
rate_5 = complete_rate(data_drop_5)
print(rate_5)

108195 records of the original edX dataset remain if we use suppression to make it 5-anonymous
                            completed
course_id                            
HarvardX/1368.1x/2T2016      0.000000
HarvardX/1368x/2T2017        0.000000
HarvardX/AI12.1x/2013_SOND   0.000000
HarvardX/AI12.2x/2013_SOND   0.000000
HarvardX/AT1x/2T2014         0.008013
...                               ...
Harvardx/HLS2X/4T2017        0.012346
Harvardx/HLS2X/T12016        0.002959
VJx/VJx/3T2014               0.000000
VJx/VJx_2/3T2016             0.000000
VJx/VJx_S/3T2015             0.000000

[215 rows x 1 columns]


In [12]:
# baseline completion rate
rate_base = complete_rate(data)
print(rate_base)

                         completed
course_id                         
HarvardX/1368.1x/2T2016   0.033439
HarvardX/1368.1x/3T2014   0.064228
HarvardX/1368.2x/2T2015   0.063197
HarvardX/1368.2x/2T2016   0.093750
HarvardX/1368.3x/2T2015   0.080645
...                            ...
Harvardx/HLS2X/T12016     0.088883
VJx/VJx/3T2014            0.078947
VJx/VJx/3T2015            0.089744
VJx/VJx_2/3T2016          0.044908
VJx/VJx_S/3T2015          0.029350

[280 rows x 1 columns]


In [13]:
print(np.mean(rate_base.completed))
print(np.mean(rate_3.completed))
print(np.mean(rate_4.completed))
print(np.mean(rate_5.completed))

0.039608028785280684
0.006129202609986929
0.00439485648184335
0.003453202330136351


> Since there are more than 200 courses, the completion rate for each course seem a little hard to compare with each other, we further calculate the average values of those courses. And we found that for the original dataset, the average completion rate for each of the course is 3.96%. For 3-anonymous dataset, the average completion rate for each of the course is 0.6%. For 4-anonymous dataset, the average completion rate for each of the course is 0.4%. For 5-anonymous dataset, the average completion rate for each of the course is 0.3%.

In [14]:
print(data[["completed"]].sum() / data[["completed"]].count())
print(data_drop_3[["completed"]].sum() / data_drop_3[["completed"]].count())
print(data_drop_4[["completed"]].sum() / data_drop_4[["completed"]].count())
print(data_drop_5[["completed"]].sum() / data_drop_5[["completed"]].count())

completed    0.024648
dtype: float64
completed    0.001799
dtype: float64
completed    0.001422
dtype: float64
completed    0.001202
dtype: float64


> Furthermore, we also calculate the overall completion rate for further comparision. And we found that for the original dataset, the overall completion rate for each of the course is 2.36%. For 3-anonymous dataset, the average completion rate for each of the course is 0.18%. For 4-anonymous dataset, the average completion rate for each of the course is 0.14%. For 5-anonymous dataset, the average completion rate for each of the course is 0.12%.

### Task 2 Synthetic K-Anonymity

In [15]:
# adding synthetic records
def synthetic_records(data, quasi_set, k):
    len_add = 0
    grouped = data.groupby(by=quasi_set)
    for name, group in grouped:
        if group.shape[0] < k:
            len_add +=  (k - group.shape[0])
    return len_add

In [16]:
synthetic_records(data_fillna, quasi_lists, 3)

1924273

> This shows we have to add 1924273 records to the original edX dataset to make it 3-anonymous.

In [17]:
synthetic_records(data_fillna, quasi_lists, 4)

2905944

> This shows we have to add 2905944 records to the original edX dataset to make it 3-anonymous.

In [18]:
synthetic_records(data_fillna, quasi_lists, 5)

3890815

> This shows we have to add 3890815 records to the original edX dataset to make it 3-anonymous.

### Task 3 - Generalization or Blurring

In [19]:
quasi_df = data_fillna[quasi_lists]
quasi_df.head()

,course_id,cc_by_ip,countryLabel,continent,city,region,subdivision,postalCode,LoE,YoB,gender,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events,mode,email_domain
0,HarvardX/PH525.1x/1T2018,GB,United Kingdom,Europe,Middlesbrough,MDB,Middlesbrough,-999,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,audit,hotmail.co.uk
1,HarvardX/PH525.1x/1T2018,US,United States,Americas,Seattle,WA,Washington,98105,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,audit,gmail.com
2,HarvardX/PH525.1x/1T2018,-999,-999,-999,-999,-999,-999,-999,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,audit,gmail.com
3,HarvardX/PH525.1x/1T2018,US,United States,North America,-999,-999,-999,-999,-999,-999.0,-999,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,audit,hotmail.com
4,HarvardX/PH525.1x/1T2018,HU,Hungary,Europe,Budapest,BU,Budapest fovaros,-999,b,1986.0,m,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,audit,gmail.com


In [20]:
quasi_df.describe()

,YoB,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events
count,1.141735e+06,1.141735e+06,1.141735e+06,1.141735e+06,1.141735e+06,1.141735e+06,1.141735e+06,1.141735e+06
mean,1.527779e+03,-9.643190e+02,-9.644928e+02,-9.645596e+02,-9.644883e+02,-9.643913e+02,-9.645603e+02,-2.316902e+02
std,1.074101e+03,1.835682e+02,1.826540e+02,1.822660e+02,1.826455e+02,1.831812e+02,1.822620e+02,4.275244e+02
min,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02
25%,1.969000e+03,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,0.000000e+00
50%,1.985000e+03,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,0.000000e+00
75%,1.992000e+03,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,0.000000e+00
max,3.116000e+03,1.091000e+03,2.507000e+03,7.800000e+01,1.420000e+02,1.024000e+03,7.700000e+01,1.782900e+04


In [21]:
quasi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1141735 entries, 0 to 1141734
Data columns (total 20 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   course_id        1141735 non-null  object 
 1   cc_by_ip         1141735 non-null  object 
 2   countryLabel     1141735 non-null  object 
 3   continent        1141735 non-null  object 
 4   city             1141735 non-null  object 
 5   region           1141735 non-null  object 
 6   subdivision      1141735 non-null  object 
 7   postalCode       1141735 non-null  object 
 8   LoE              1141735 non-null  object 
 9   YoB              1141735 non-null  float64
 10  gender           1141735 non-null  object 
 11  nforum_posts     1141735 non-null  float64
 12  nforum_votes     1141735 non-null  float64
 13  nforum_endorsed  1141735 non-null  float64
 14  nforum_threads   1141735 non-null  float64
 15  nforum_comments  1141735 non-null  float64
 16  nforum_pinned    1

In [22]:
df = data_fillna.copy()

In [23]:
categorical = ['course_id', 'cc_by_ip', 'countryLabel', 'continent', 'city', 'region',
                  'subdivision', 'postalCode', 'LoE', 'gender', 'mode', 'email_domain']

numerical = ['YoB','nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments',
            'nforum_pinned', 'nforum_events']

In [24]:
for name in categorical:
    df[name] = df[name].astype('category')

Next we take a look at spans (max-min for numerical columns, number of different values for categorical columns) of all columns for a partition of a dataframe.

In [25]:
## spans (max-min for numerical columns, number of different values for categorical columns) 
## of all columns for a partition of a dataframe.

def get_spans(df, partition, scale=None):
    spans = {}
    for column in df.columns:
        if column in categorical:
            span = len(df[column][partition].unique())
        elif column in numerical:
            span = df[column][partition].max()-df[column][partition].min()
        else:
            continue
        if scale is not None:
            span = span/scale[column]
        spans[column] = span
    return spans

In [26]:
full_spans = get_spans(df, df.index)
print(full_spans)

{'course_id': 280, 'cc_by_ip': 226, 'countryLabel': 244, 'continent': 9, 'city': 26138, 'region': 1084, 'subdivision': 2508, 'postalCode': 33174, 'LoE': 12, 'YoB': 4115.0, 'gender': 4, 'nforum_posts': 2090.0, 'nforum_votes': 3506.0, 'nforum_endorsed': 1077.0, 'nforum_threads': 1141.0, 'nforum_comments': 2023.0, 'nforum_pinned': 1076.0, 'nforum_events': 18828.0, 'mode': 4, 'email_domain': 45430}


Next, we take a look at all categorical variables, and check to see whether we should generalize or delete them, or leave it there.

In [27]:
for i in categorical:
    counts = df[i].value_counts()
    filtered = counts[counts <= 5]
    print(counts)
    print("number of values with <5 counts is %i"%len(filtered))
    print("=================================")

HarvardX/CS50x3/2015                        126249
HarvardX/CS50/X                             124119
HarvardX/CS50x/2014_T1                       70600
HarvardX/GSD1x/1T2017                        31917
HarvardX/MCB80.1x/2013_SOND                  19616
                                             ...  
HarvardX/HLS1xC/Copyright                       27
HarvardX/MUS24.4x/2T2018                        27
HarvardX/HLS1xA/Copyright                       26
HarvardX/BUS5.1x_Application_Only/3T2015        18
HarvardX/HLS1x/2013_Spring                       4
Name: course_id, Length: 280, dtype: int64
number of values with <5 counts is 1
US      308402
-999    184426
IN       77301
GB       40781
BR       38065
         ...  
PW           3
ST           2
AQ           1
FK           1
SM           1
Name: cc_by_ip, Length: 226, dtype: int64
number of values with <5 counts is 11
United States                  308402
-999                           184195
India                           77301


* 'course_id': number of values with <5 counts is 1, leave it there for checking completion rate

* 'cc_by_ip': number of values with <5 counts is 11, leave it there. We could remove those records in surpression algorithm. It would be taking too much computation power and time consuming to generalize, and it may lose too much info if we blurring and delete. I will keep it there.


* 'countryLabel': number of values with <5 counts is 16, leave it there. We could remove those records in surpression algorithm. It would be taking too much computation power and time consuming to generalize, and it may lose too much info if we blurring and delete. I will keep it there.


* 'continent': number of values with <5 counts is 1, leave it there. We could remove those records in surpression algorithm. It would be taking too much computation power and time consuming to generalize, and it may lose too much info if we blurring and delete. I will keep it there.


* 'city': number of values with <5 counts is 17436, which is large, maybe consider blurring this field would be better. 

* 'region': number of values with <5 counts is 170, which is okay, we could remove those records in surpression algorithm. It would be taking too much computation power and time consuming to generalize, and it may lose too much info if we blurring and delete. I will keep it there.

* 'subdivision': number of values with <5 counts is 653, which is okay, we could remove those records in surpression algorithm. It would be taking too much computation power and time consuming to generalize, and it may lose too much info if we blurring and delete. I will keep it there.

* 'postalCode': number of values with <5 counts is 20194, which is large, maybe consider blurring this field would be better. 
* 'LoE': number of values with <5 counts is 0, we could keep this variable there without doing anything as it already have a good level of anonmity and generalization in this column

* 'gender': number of values with <5 counts is 0, we could keep this variable there without doing anything as it already have a good level of anonmity and generalization in this column

* 'mode': number of values with <5 counts is 0, we could keep this variable there without doing anything as it already have a good level of anonmity and generalization in this column

* 'email_domain': number of values with <5 counts is 42507, maybe consider blurring this field would be better. 

In [28]:
categorical_blurring = ['city','postalCode', 'LoE', 'email_domain']

For all numerical columns, we could try to generalize it and divide them into sub categories and partitions, which will be written in details in **generalize** function.

* 'YoB':  partition and split into different year range based on a 10 year interval (50s, 60s, 70s etc.)
* 'nforum_posts': partition and split into different bins according to quantile
* 'nforum_votes': same as above
* 'nforum_endorsed': same as above
* 'nforum_threads': same as above
* 'nforum_comments': same as above
* 'nforum_pinned': same as above
* 'nforum_events': same as above

In [29]:
# def split(df, partition, column):
#     dfp = df[column][partition]
#     if column in categorical:
#         values = dfp.unique()
#         lv = set(values[:len(values)//2])
#         rv = set(values[len(values)//2:])
#         return dfp.index[dfp.isin(lv)], dfp.index[dfp.isin(rv)]
#     elif column in numerical:        
#         median = dfp.median()
#         dfl = dfp.index[dfp < median]
#         dfr = dfp.index[dfp >= median]
#         return (dfl, dfr)

### Python Code for Blurring and Generalize

In [30]:
# blurring (column suppression, removing whole columns that don't fit for k-anonymity) 
def blurring(df, col):
    result_df = df.drop(columns=[col])
    return result_df

In [31]:
# generalization (changing column values to be more general such as using an age range instead of a specific age)
def generalize(df, col, partition = 4):
    result_df = df.copy()
    
    ## generalize YoB into Bins
    if col == "YoB":
        result_df['age'] = pd.cut(result_df.YoB, [-1000, 1950, 1960,1970,1980,1990,2000,2010,2020, max(result_df.YoB)+1], 
               labels = ["50s-", "50s", '60s','70s','80s','90s','00s','10s','20s+']).astype(str)
    
    ## if col is numerical other than YoB, generalize them into sub-bins by quantiles
    elif col in numerical: 
        #result_df.sort_values(by =[col], inplace = True)
        #result_df[str(col+"_percentile")] = pd.qcut(result_df[col], q = partition, labels = False, duplicates = "raise") 
        span = max(result_df[col]) - 0
        interval = 1/partition
        partition_lst = [i*interval*span for i in range(partition)]
        label_lst = ['q'+str(i) for i in range(partition)]
        pd_cut_full_lst, label_full_lst = [-1000], ["q1-"]
        pd_cut_full_lst += partition_lst
        label_full_lst += label_lst
        pd_cut_full_lst.append(max(result_df[col])+1)
        #print(pd_cut_full_lst)
        #print(label_full_lst)
        result_df[str(col+"_percentile")] = pd.cut(result_df[col], pd_cut_full_lst, labels = label_full_lst).astype(str)
    
    ## last step: remove original column
    result_df = result_df.drop(columns=[col])
    return result_df

### Examples

In [32]:
df1= blurring(df, "city")

In [33]:
# deal with YoB
df1 = generalize(df, "YoB")
df1['age']

0          50s-
1          50s-
2          50s-
3          50s-
4           80s
           ... 
1141730    50s-
1141731     90s
1141732    50s-
1141733     80s
1141734     90s
Name: age, Length: 1141735, dtype: object

In [34]:
df1.head()

,course_id,user_id,username,registered,viewed,explored,certified,completed,ip,cc_by_ip,...,roles_isStaff,roles_isCCX,roles_isFinance,roles_isLibrary,roles_isSales,forumRoles_isAdmin,forumRoles_isCommunityTA,forumRoles_isModerator,forumRoles_isStudent,age
0,HarvardX/PH525.1x/1T2018,1488411,KIRSTEN SUAREZ,True,False,-999,False,False,81.108.107.58,GB,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,50s-
1,HarvardX/PH525.1x/1T2018,7013084,CAREY FOSTER,True,True,True,False,False,205.175.107.76,US,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,50s-
2,HarvardX/PH525.1x/1T2018,4083257,CLAUDINE FARMER,True,True,False,False,False,103.212.146.137,-999,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,50s-
3,HarvardX/PH525.1x/1T2018,4605571,SHEREE BONNER,True,True,False,False,False,172.221.204.94,US,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,50s-
4,HarvardX/PH525.1x/1T2018,1499820,MITCHELL VALDEZ,True,True,True,False,False,193.225.200.92,HU,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,80s


In [35]:
# deal with nForum
df1 = generalize(df, "nforum_posts")
df1['nforum_posts_percentile'].describe()

count     1141735
unique          5
top           q1-
freq      1102375
Name: nforum_posts_percentile, dtype: object

In [36]:
# test suppression on new df1
len_remove, lst_remove = supression(df1, quasi_lists[:3], 5)
print(len_remove)
data_drop_3_age = data.drop(lst_remove, axis=0)
rate_3_age = complete_rate(data_drop_3_age)
print(rate_3_age)

30069
                         completed
course_id                         
HarvardX/1368.1x/2T2016   0.033592
HarvardX/1368.1x/3T2014   0.061265
HarvardX/1368.2x/2T2015   0.064698
HarvardX/1368.2x/2T2016   0.102389
HarvardX/1368.3x/2T2015   0.081613
...                            ...
Harvardx/HLS2X/T12016     0.086794
VJx/VJx/3T2014            0.080372
VJx/VJx/3T2015            0.095238
VJx/VJx_2/3T2016          0.045650
VJx/VJx_S/3T2015          0.029367

[279 rows x 1 columns]


### Driver script repeatedly search with different generalization and blurring choices, to achieve 5-anonmity

In [37]:
def naive_search(col_lst, df):
    remove_dict = {}
    total_completion_rate_dict = {}
    for i in col_lst:
        quasi_lists_copy = quasi_lists.copy()
        if i in numerical:
            temp_df = generalize(df, i)
            # new quasi_lists after generalize
            print(quasi_lists_copy)
            quasi_lists_copy.remove(i)
            if i == "YoB":
                quasi_lists_copy.append("age")
            else:
                quasi_lists_copy.append(str(i+"_percentile"))
            len_remove, lst_remove = supression(temp_df, quasi_lists_copy, 5)
            print("if generalize on "+ i + " we need to remove "+ str(len_remove) + " records.")
        elif i in categorical_blurring:
            temp_df = blurring(df, i)
            # new quasi_lists after blurring
            quasi_lists_copy.remove(i)
            len_remove, lst_remove = supression(temp_df, quasi_lists_copy, 5)
            print("if blurring on "+ i + " we need to remove "+ str(len_remove) + " records.")
        data_drop= temp_df.drop(lst_remove, axis=0)
        rate = complete_rate(data_drop)
        total_rate = (data_drop[["completed"]].sum()) / (data_drop[["completed"]].count())
        print("total completion rate is : %.4f"%total_rate)
        remove_dict[i] = len_remove
        total_completion_rate_dict[i] = total_rate
        print("===========================================")
    return temp_df

In [38]:
search_lst = categorical_blurring + numerical
search_lst

['city',
 'postalCode',
 'LoE',
 'email_domain',
 'YoB',
 'nforum_posts',
 'nforum_votes',
 'nforum_endorsed',
 'nforum_threads',
 'nforum_comments',
 'nforum_pinned',
 'nforum_events']

In [39]:
naive_search(search_lst, df)
#naive_search(["'YoB', "nforum_posts"], df)

if blurring on city we need to remove 1028395 records.
total completion rate is : 0.0013
if blurring on postalCode we need to remove 1024271 records.
total completion rate is : 0.0013
if blurring on LoE we need to remove 994799 records.
total completion rate is : 0.0014
if blurring on email_domain we need to remove 973672 records.
total completion rate is : 0.0016
['course_id', 'cc_by_ip', 'countryLabel', 'continent', 'city', 'region', 'subdivision', 'postalCode', 'LoE', 'YoB', 'gender', 'nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', 'nforum_events', 'mode', 'email_domain']
if generalize on YoB we need to remove 964866 records.
total completion rate is : 0.0018
['course_id', 'cc_by_ip', 'countryLabel', 'continent', 'city', 'region', 'subdivision', 'postalCode', 'LoE', 'YoB', 'gender', 'nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', 'nforum_events', 'mode', 'email_domain']


,course_id,user_id,username,registered,viewed,explored,certified,completed,ip,cc_by_ip,...,roles_isStaff,roles_isCCX,roles_isFinance,roles_isLibrary,roles_isSales,forumRoles_isAdmin,forumRoles_isCommunityTA,forumRoles_isModerator,forumRoles_isStudent,nforum_events_percentile
0,HarvardX/PH525.1x/1T2018,1488411,KIRSTEN SUAREZ,True,False,-999,False,False,81.108.107.58,GB,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,q1-
1,HarvardX/PH525.1x/1T2018,7013084,CAREY FOSTER,True,True,True,False,False,205.175.107.76,US,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,q1-
2,HarvardX/PH525.1x/1T2018,4083257,CLAUDINE FARMER,True,True,False,False,False,103.212.146.137,-999,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,q1-
3,HarvardX/PH525.1x/1T2018,4605571,SHEREE BONNER,True,True,False,False,False,172.221.204.94,US,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,q1-
4,HarvardX/PH525.1x/1T2018,1499820,MITCHELL VALDEZ,True,True,True,False,False,193.225.200.92,HU,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,q1-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141730,HarvardX/PH525.5x/2T2018,1388822,WILLIE HOGAN,True,False,-999,False,False,-999,-999,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,q1-
1141731,HarvardX/PH525.5x/2T2018,9210079,ESTELA WRIGHT,True,False,-999,False,False,59.180.173.247,IN,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,q1-
1141732,HarvardX/PH525.5x/2T2018,6212000,NONA HALEY,True,False,-999,False,False,124.17.34.78,CN,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,q1-
1141733,HarvardX/PH525.5x/2T2018,945808,JULIO SANDOVAL,True,False,-999,False,False,191.177.186.50,BR,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0,q1-


From the above search we could see that, if we only search for manipulating minimal number of columns (which is 1 in this case, as we didn't do an exhaustive grid search on all permutations of columns for generalization or blurring), the results are shown above.

If we want highest total completion rate, we should **generalize on YoB**, which leads to 0.0018 completion rate when we suppress and get 5-anonymity 

If we want to remove least records, we should also choose to **generalize on YoB**, which only needs to remove 964866 records for suppression to get 5-anonymity.

**Summary: **

Based on the approach we adopted, we first learned from the data generated in Step 3 that different generalization and blurring choices result in output that varies. This makes subjective choices necessary in the process of de-identification both in terms of selecting the statistical techniques for de-identification and determining the generated data set that best represents the original dataset. 

Second, we realized from this practice that regardless of the techniques we use, the process of de-identification will inevitably change the structure and nature of the original dataset and make the final de-identified dataset less accurate. For instance, after generalization on YoB for realizing 5-anonymity, the total completion rate is 0.18%, which is wildly different from what we would have gotten from the original dataset, which is 4%. Admittedly, this statistics (and others generated from our de-identified datasets) could have been improved if we did an exhaustive grid search on all permutations of columns for generalization or blurring. Indeed, this recognized limitation leads to the last lesson we learned from the Step 3 exercise: Generating high quality de-identified dataset that is meaningful and useful for future research is a laborious but rewarding task that requires researchers to be wholeheartedly devoted to advancing their technical skills and fulfilling their ethical obligations.

### Task 6 - I-Diversity

In [40]:
sensitive_variables = ["completed" , 'grade', 'cert_status']

In [41]:
df_new_after_step_3 = naive_search(["YoB"], df)

['course_id', 'cc_by_ip', 'countryLabel', 'continent', 'city', 'region', 'subdivision', 'postalCode', 'LoE', 'YoB', 'gender', 'nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', 'nforum_events', 'mode', 'email_domain']
if generalize on YoB we need to remove 964866 records.
total completion rate is : 0.0018


In [42]:
for var in sensitive_variables:
    i_diversity = df_new_after_step_3[var].value_counts()
    print("sensitive variable name: "+var)
    print(i_diversity)
    print("=============================================")

sensitive variable name: completed
False    1113593
True       28142
Name: completed, dtype: int64
sensitive variable name: grade
 0.00      727322
-999.00    321067
 0.01        6491
 0.03        5933
 0.02        4756
            ...  
 1.05           3
 1.19           1
 1.16           1
 1.11           1
 1.15           1
Name: grade, Length: 114, dtype: int64
sensitive variable name: cert_status
-999                614558
notpassing          487232
downloadable         22675
audit_notpassing     16582
audit_passing          447
unverified             220
unavailable             20
error                    1
Name: cert_status, dtype: int64


In [43]:
new_quasi_lists =  ['course_id', 'cc_by_ip', 'countryLabel', 'continent', 'city', 'region',
              'subdivision', 'postalCode', 'LoE', 'age', 'gender', 'nforum_posts', 'nforum_votes', 
              'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', 'nforum_events', 
              'mode', 'email_domain']

In [44]:
# take l-diveristy for each group divided by new quasi list

grouped = df_new_after_step_3.groupby(by=new_quasi_lists)

l_diversity =  {"completed":{}, 'grade':{}, 'cert_status':{}}

for name, group in grouped:
    for var in sensitive_variables:

        l_div = group[var].nunique()
        if l_div in l_diversity[var].keys():
            l_diversity[var][l_div] += 1
        else:
            l_diversity[var][l_div] = 1

In [45]:
l_diversity

{'completed': {1: 908040, 2: 893},
 'grade': {1: 898097,
  2: 9887,
  5: 55,
  3: 728,
  4: 128,
  6: 23,
  8: 3,
  9: 5,
  14: 1,
  7: 2,
  11: 2,
  10: 2},
 'cert_status': {1: 901483, 2: 7364, 3: 86}}

If we only look at minimal *l-diversity* in all block for three variables:

**all of them acheived only 1-diversity**

In [46]:
[min(l_diversity[var].keys()) for var in sensitive_variables]

[1, 1, 1]

### Testing

> For testing, we would test our blurring and generalizing algorithm to make sure the code we write to deal with the dataframe will produce the right answer we expected when it ran to completion.
We won't do exhaustive testing or unit test for every single line we wrote here like what traditional software development do. Below is a demo for testing.

> If we have more time to improve, testing should cover different aspects, algorithms and possible columns on a subset of dataframe, to make sure every mechanism works fine.

In [47]:
# create a sample sub dataframe for testing
test_df = df[:10]
test_df

,course_id,user_id,username,registered,viewed,explored,certified,completed,ip,cc_by_ip,...,roles_isInstructor,roles_isStaff,roles_isCCX,roles_isFinance,roles_isLibrary,roles_isSales,forumRoles_isAdmin,forumRoles_isCommunityTA,forumRoles_isModerator,forumRoles_isStudent
0,HarvardX/PH525.1x/1T2018,1488411,KIRSTEN SUAREZ,True,False,-999,False,False,81.108.107.58,GB,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0
1,HarvardX/PH525.1x/1T2018,7013084,CAREY FOSTER,True,True,True,False,False,205.175.107.76,US,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0
2,HarvardX/PH525.1x/1T2018,4083257,CLAUDINE FARMER,True,True,False,False,False,103.212.146.137,-999,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0
3,HarvardX/PH525.1x/1T2018,4605571,SHEREE BONNER,True,True,False,False,False,172.221.204.94,US,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0
4,HarvardX/PH525.1x/1T2018,1499820,MITCHELL VALDEZ,True,True,True,False,False,193.225.200.92,HU,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0
5,HarvardX/PH525.1x/1T2018,3872821,CLYDE FINCH,True,True,False,False,False,177.18.230.216,BR,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0
6,HarvardX/PH525.1x/1T2018,7560679,MITZI AVERY,True,True,True,False,False,73.153.215.76,US,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0
7,HarvardX/PH525.1x/1T2018,18306,IRIS MARQUEZ,True,False,-999,False,False,2.84.185.183,GR,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0
8,HarvardX/PH525.1x/1T2018,6530939,LYNETTE GRAHAM,True,False,-999,False,False,129.21.116.121,US,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0
9,HarvardX/PH525.1x/1T2018,8874860,KATE ROWLAND,True,False,-999,False,False,77.245.2.66,JO,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.0


In [48]:
test1 = supression(test_df, quasi_lists[:3], 3)
# by manually grouping the dataset and check the result
assert test1[0] == 6
assert test1[1] == [2, 5, 0, 7, 4, 9]

In [49]:
test2 = supression(test_df, quasi_lists[6:10], 4)
# by manually grouping the dataset and check the result
assert test2[0] == 6
assert test2[1] == [7, 4, 0, 8, 5, 1]

In [50]:
test3 = synthetic_records(test_df, quasi_lists[6:10], 4)
# by manually grouping the dataset and check the result
assert test3 == 18

In [51]:
test4 = synthetic_records(test_df, quasi_lists[:3], 5)
# by manually grouping the dataset and check the result
assert test4 == 25

In [52]:
def test_blurring(test_df, col):
    result = blurring(test_df, col)
    expected = test_df.drop(col, 1)
    assert result.equals(expected)

test_blurring(test_df, "registered")

In [53]:
def test_generalize_nforum(full_df, test_df, col):
    result = generalize(full_df, col)
    result = result[:10]
    expected = test_df.copy()
    expected[str(col+"_percentile")] = "q1-"
    expected = expected.drop(columns=[col])
    assert result.equals(expected)

test_generalize_nforum(df, test_df, "nforum_posts")

> Our algorithm work well and passed testing by not throwing any assertion errors.

### Summary:
    
We defined “completed” (indicated that the user completed the course by achieving a final grade above the overall passing threshold for the course), “grade” (Final grade earned in the course, on a scale of 0 to 1), and “cert_status” (certification status, specified as ‘downloadable’, ‘notpassing’, ‘unavailable’, ‘audit_notpassing’, ‘audit_passing’ or ‘error’) as sensitive attributes. We made this decision based on the FERPA in which statistics related to student grades are protected.  Due to the special features of online learning platforms such as edX, we considered both a user’s completion status and certification status as sensitive information in addition to his/her grades. We proceeded our analysis with a dataset that generalizes on YoB because, based on the analysis from Step 3, this dataset has the total completion rate that is most close to the baseline rate and includes the most number of tuples.

Descriptive analysis of the count of the values of **“completed”** reveals that this sensitive attribute is extremely vulnerable to the homogeneity attack because it only contains two values (‘True’ or ‘False’). In this case, for each q*-block that includes 5 tuples whose values for all the quasi-identifiers are identical, it is very likely that they have the same value for the attribute “completed” due to its lack of diversity. This means that knowing a person belonging to a certain q*-block without identifying exactly which record in this q*-block represents that person is sufficient for an adversary to obtain information on his/her completion of the course. Further analysis on the diversity of each q*-block of the dataset indicates that 908040 of the total blocks of tuples have an l-diversity of 1, putting an individual’s information in danger of being leaked.The rest q*-blocks have an l-diversity of 2, protecting an individual from homogeneity attack. Achieving a higher level of l-diversity for this attribute would mean to manipulate the data so that a give q*-block contains two values and their frequencies of occurrence are roughly balanced.

Descriptive analysis of the count of the values of **“grade”** reveals that this sensitive attribute is moderately vulnerable to the homogeneity attack and background knowledge because 1) the distribution of the grades is heavily left-skewed and 2) a person’s grade is highly related to their prior academic performance, information that is likely to be obtained by an adversary. Firstly, for each q*-block that includes 5 tuples whose values for all the quasi-identifiers are identical, it is very likely that they all have the most frequent grade (i.e., ‘0.00’) or grades that are less frequent but the amounts of occurrences are still substantial (e.g., ‘0.01’, ‘0.02’, ‘0.03’). This means that knowing a person belonging to a certain q*-block without identifying exactly which record in this q*-block represents that person is sufficient for an adversary to obtain information on his/her grade. Further analysis on the diversity of each q*-block of the dataset indicates that 898097 of the total blocks of tuples have an l-diversity of 1, putting an individual’s information in danger of being leaked.The rest q*-blocks have an l-diversity of 2 or more, protecting an individual from homogeneity attack but not necessarily protecting him/her from background knowledge. For instance, if a q*-block with 5 tuples includes only one tuple that has a grade of 0, with all the other tuples having a grade of 1, this would be a 2-diversity q*-block that is supposed to be sufficient to protect an individual's information. However, if an adversary who knows a certain person belonging to this given q*-block and manages to glean information from external datasets indicating that this person has poor school performance, it is reasonable for the adversary to conclude that this person’s grade for a given course is 0. Achieving a higher level of l-diversity for this attribute would thus mean to manipulate the data so that a give q*-block contains more than two values of the attribute so there is a reasonable range of grades and their frequencies of occurrence are roughly balanced. 

Descriptive analysis of the count of the values of **“cert_status”** reveals that this sensitive attribute is moderately vulnerable to the homogeneity attack and background knowledge because 1) the distribution of the status is heavily left-skewed and 2) a person’s audit/enrollment status is to some extent related to their professions, information that is likely to be obtained by an adversary. Firstly, for each q*-block that includes 5 tuples whose values for all the quasi-identifiers are identical, it is very likely that they all have the most frequent status (i.e., ‘not passing’) or status that is less frequent but the amounts of occurrences are still substantial (e.g., ‘downloadable’, ‘audit_notpassing’). This means that knowing a person belonging to a certain q*-block without identifying exactly which record in this q*-block represents that person is sufficient for an adversary to obtain information on his/her certification status. Further analysis on the diversity of each q*-block of the dataset indicates that 901483 of the total blocks of tuples have an l-diversity of 1, putting an individual’s information in danger of being leaked.The rest q*-blocks have an l-diversity of 2 or more, protecting an individual from homogeneity attack but not necessarily protecting him/her from background knowledge. For instance, if a q*-block with 5 tuples includes only one tuple that has a status ‘audit_notpassing’, with all the other tuples having ‘passing’,  this would be a 2-diversity q*-block that is supposed to be sufficient to protect an individual's information. However, if an adversary who knows a certain person belonging to this given q*-block and manages to glean information from external datasets indicating that this person has a busy work/study schedule, it is reasonable for the adversary to conclude that this person is very likely to audit the class and since there is only one value for the option audit, the adversary has enough confidence to conclude that this person is not passing. Achieving a higher level of l-diversity for this attribute would thus mean to manipulate the data so that a give q*-block contains at least two values for both the audit and enrollee options (i.e., an l-diversity of at least 4) and their frequencies of occurrence are roughly balanced. 